# Important Configurations

In [ ]:
from config import open_ai_key
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
import os

embedding_model = OpenAIEmbeddings(api_key=open_ai_key)
model_name = "gpt-4o"
llm = ChatOpenAI(model_name=model_name, openai_api_key=open_ai_key)

os.environ["OPENAI_API_KEY"] = open_ai_key

# Load All the Documents

In [34]:
from langchain_community.document_loaders import PyMuPDFLoader
import os

pdf_folder_path = "./data"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyMuPDFLoader(pdf_path)
        documents.extend(loader.load())
        
print(f"total documents loaded - {len(documents)}")

# Chunking Strategies

## Fixed Length Chunking

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
separator = "\n\n",
chunk_size = 1000,
chunk_overlap  = 128
)
chunked_documents  = text_splitter.split_documents(documents)

## Recursive Chunking

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 1000,
chunk_overlap  = 128
)

chunked_documents  = text_splitter.split_documents(documents)


## Semantic Chunking

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(embedding_model)

chunked_documents  = text_splitter.split_documents(documents)

## Agentic Chunking

In [75]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub

In [77]:
proposition_prompt = hub.pull("wfh/proposal-indexing")
proposition_prompt.messages[1].prompt.template = proposition_prompt.messages[1].prompt.template + "\n\n" + "This data is regarding HR Policies of India. Generate Proposistions and do decompositions accordingly"
runnable = proposition_prompt | llm

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
import concurrent.futures


class Sentences(BaseModel):
    sentences: List[str] = Field(
        description="These are the paragraphs from an HR policy"
    )


extraction_chain = llm.with_structured_output(Sentences)


def get_propositions(text):
    runnable_output = runnable.invoke({"input": text}).content
    propositions = extraction_chain.invoke(runnable_output.sentences)
    return propositions


essay_propositions = []


def process_paragraph(para):
    """Extracts propositions from a paragraph."""
    propositions = get_propositions(para.page_content)
    return propositions.sentences

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = {
        executor.submit(process_paragraph, para): i for i, para in enumerate(documents)
    }

    for future in concurrent.futures.as_completed(futures):
        i = futures[future]  # Get the paragraph index
        try:
            essay_propositions.extend(future.result())
        except Exception as e:
            print(f"Error processing paragraph {i}: {e}")
        print(f"Done with {i}")
        break

Error processing paragraph 3: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-MleP4eNCIIMjcaTNaMI7A9vQ on tokens per min (TPM): Limit 30000, Used 29001, Requested 1513. Please try again in 1.028s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Done with 3


KeyboardInterrupt: 

# Vector Storage

In [6]:
from langchain.vectorstores import Chroma


def create_or_load_vector_db(chunked_documents, persist_directory, load=True):
    if load:
        vectordb = Chroma(
            persist_directory=persist_directory,
            embedding_function=OpenAIEmbeddings(api_key=open_ai_key),
        )

    else:
        vectordb = Chroma.from_documents(
            documents=chunked_documents,
            embedding=embedding_model,
            persist_directory=persist_directory,
        )
    
    return vectordb

vectordb = create_or_load_vector_db(chunked_documents, "./vector_store_agentic", True)

/tmp/ipykernel_23242/3852689682.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings(api_key=open_ai_key),
/tmp/ipykernel_23242/3852689682.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="vector_store_semantic", embedding_function=OpenAIEmbeddings(api_key=open_ai_key))


# Creating a Retrieval & Generation

In [74]:
from langchain.chains.question_answering import load_qa_chain

def create_agent_chain():
    chain = load_qa_chain(llm, chain_type="stuff")
    return llm

def get_llm_response(query, vectordb):
    chain = create_agent_chain()
    matching_docs = vectordb.similarity_search(query)
    context = [matching_docs[0].page_content, matching_docs[1].page_content, matching_docs[2].page_content]
    prompt = f"You are a HR policies Expert. Based on the provided context from policies, answer the user question.\Context: {[' '.join(c) for c in context]}\nQuestion: {query}\nAnswer as if you are helpful person answering the user.  Give a concise answer only"
    answer = chain.invoke(prompt)
    return answer.content, context

In [3]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub

/home/himanshu-singh/anaconda3/envs/rag_advanced/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
obj = hub.pull("wfh/proposal-indexing")
llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key = os.environ["OPENAI_API_KEY"])

/home/himanshu-singh/anaconda3/envs/rag_advanced/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [5]:
obj.messages[1].prompt.template = obj.messages[1].prompt.template + "\n\n" + "This data is regarding HR Policies of India. Generate Proposistions and do decompositions accordingly"

print(obj.messages[1].prompt.template)

Decompose the following:
{input}

This data is regarding HR Policies of India. Generate Proposistions and do decompositions accordingly


In [6]:
runnable = obj | llm

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Sentences(BaseModel):
    sentences: List[str] = Field(description="These are the paragraphs from an HR policy")
    
# Extraction
extraction_chain = llm.with_structured_output(Sentences)

/home/himanshu-singh/anaconda3/envs/rag_advanced/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1369: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [8]:
def get_propositions(text):
    runnable_output = extraction_chain.invoke(text)
    propositions = extraction_chain.invoke(runnable_output.sentences)
    return propositions

In [9]:
import concurrent.futures

essay_propositions = []

def process_paragraph(para):
    """Extracts propositions from a paragraph."""
    propositions = get_propositions(para.page_content)
    return propositions.sentences

# Using ThreadPoolExecutor with 5 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(process_paragraph, para): i for i, para in enumerate(documents)}
    
    for future in concurrent.futures.as_completed(futures):
        i = futures[future]  # Get the paragraph index
        try:
            essay_propositions.extend(future.result())
        except Exception as e:
            print(f"Error processing paragraph {i}: {e}")
        print(f"Done with {i}")


Done with 3
Done with 1
Done with 2
Done with 4
Done with 5
Done with 0
Done with 6
Done with 7
Done with 8
Done with 9
Done with 10
Done with 12
Done with 11
Done with 13
Done with 17
Done with 16
Done with 15
Done with 14
Done with 18
Done with 19
Done with 21
Done with 22
Done with 20
Done with 23
Done with 24
Done with 27
Done with 25
Done with 29
Done with 26
Done with 28
Done with 31
Done with 32
Done with 30
Done with 33
Done with 37
Done with 34
Done with 36
Done with 38
Done with 35
Done with 39
Done with 40
Done with 41
Done with 42
Done with 43
Done with 44
Done with 45
Done with 50
Done with 46
Done with 48
Done with 49
Done with 47
Done with 51
Done with 53
Done with 52
Done with 56
Done with 57
Done with 59
Done with 58
Done with 60
Done with 61
Done with 62
Done with 55
Done with 63
Done with 66
Done with 67
Done with 64
Done with 54
Done with 65
Done with 68
Done with 69
Done with 70
Done with 72
Done with 75
Done with 71
Done with 74
Done with 73
Done with 77
Done with

KeyboardInterrupt: 

In [10]:
print (f"You have {len(essay_propositions)} propositions")

You have 5712 propositions


In [11]:
import json
temp_dict = {}

temp_dict["essay_propositions"] = essay_propositions

with open("essay_propositions.json", 'w') as file:
    json.dump(temp_dict, file, indent=4)

In [22]:
import concurrent.futures
import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger().setLevel(logging.ERROR)


import json
from AgenticChunker import AgenticChunker

with open("essay_propositions.json", 'r') as file:
    essay_propositions = json.load(file)
    
essay_propositions = essay_propositions["essay_propositions"]
    
# Initialize the shared AgenticChunker instance
ac = AgenticChunker()

def process_single_proposition(proposition):
    """Worker function to process a single proposition in the shared AgenticChunker instance."""
    ac.add_proposition(proposition)

def process_propositions_parallel(essay_propositions, max_workers=50):
    """
    Processes propositions in parallel using a shared AgenticChunker instance.
    Each batch contains at most `max_workers` propositions.
    """
    # Using ThreadPoolExecutor with 10 workers
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submitting the propositions in batches of `max_workers`
        for i in range(0, len(essay_propositions), max_workers):
            batch = essay_propositions[i:i + max_workers]
            futures = [executor.submit(process_single_proposition, prop) for prop in batch]

            # Wait for batch completion before submitting the next batch
            concurrent.futures.wait(futures)
            print(f"Finished {i} chunks")
    
    # Return the final chunks after all propositions are processed
    return ac.get_chunks(get_type='dict')

# Run parallel processing
final_chunks = process_propositions_parallel(essay_propositions)


Finished 0 chunks
Finished 50 chunks
Finished 100 chunks
Finished 150 chunks
Finished 200 chunks
Finished 250 chunks
Finished 300 chunks
Finished 350 chunks
Finished 400 chunks
Finished 450 chunks
Finished 500 chunks
Finished 550 chunks
Finished 600 chunks
Finished 650 chunks
Finished 700 chunks
Finished 750 chunks
Finished 800 chunks
Finished 850 chunks
Finished 900 chunks
Finished 950 chunks
Finished 1000 chunks
Finished 1050 chunks
Finished 1100 chunks
Finished 1150 chunks
Finished 1200 chunks
Finished 1250 chunks
Finished 1300 chunks
Finished 1350 chunks
Finished 1400 chunks
Finished 1450 chunks
Finished 1500 chunks
Finished 1550 chunks
Finished 1600 chunks
Finished 1650 chunks
Finished 1700 chunks
Finished 1750 chunks
Finished 1800 chunks
Finished 1850 chunks
Finished 1900 chunks
Finished 1950 chunks
Finished 2000 chunks
Finished 2050 chunks
Finished 2100 chunks
Finished 2150 chunks
Finished 2200 chunks
Finished 2250 chunks
Finished 2300 chunks
Finished 2350 chunks
Finished 2400 c

In [31]:
chunked_documents = {}
for i, (id, chunk) in enumerate(final_chunks.items()):
    final_string = ""
    final_string += f"Title: {chunk['title']}\n\n"
    final_string += f"Summary: {chunk['summary']}\n\n"
    propositions = [("").join(p) for p in chunk["propositions"]]
    final_string += f"Propositions:\n {propositions[0]}"
    chunked_documents[f"chunk_{i}"] = final_string

In [33]:
import json

with open("agentic_chunks.json", 'w') as file:
    json.dump(chunked_documents, file, indent=4)

In [ ]:
import chromadb
from langchain.vectorstores import Chroma
from config import open_ai_key
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

client = chromadb.Client()

chunked_documents = [Document(page_content=s) for i, s in enumerate(chunked_documents.values())]

vectordb = Chroma.from_documents(
documents=chunked_documents,
embedding=OpenAIEmbeddings(api_key=open_ai_key),
persist_directory="./vectofrom langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from config import open_ai_key
import json
import random

llm = ChatOpenAI(model="gpt-4o", api_key=open_ai_key)

QUESTIONS_GENERATOR = """
Given below is a context from one of the HR Policies of India.

context: {context}

Think about 5 questions that a person may ask whose answer can be given from the context

List down 5 questions in json format and json format only. Given below is reference json structure.

{{
    "question_1": {{"question": str, "answer": str}},
    "question_2": {{"question": str, "answer": str}},
    "question_3": {{"question": str, "answer": str}},
    "question_4": {{"question": str, "answer": str}},
    "question_5": {{"question": str, "answer": str}},
}}

The questions you ask, make sure that the answer can be found in the context. Make sure that tha question and answer is relevant. Dont give vague question or answer. Dont give too general questions or answers.
WHile answering the question, name the policy in detail which you are referring to

"""

class Questions(BaseModel):
question_1: dict = Field(description="First Question")
question_2: dict = Field(description="Second Question")
question_3: dict = Field(description="Third Question")
question_4: dict = Field(description="Fourth Question")
question_5: dict = Field(description="Fifth Question")

parser = JsonOutputParser(pydantic_object=Questions)

question_prompt = PromptTemplate(
template=QUESTIONS_GENERATOR,
input_variables=["context"],
partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = question_prompt | llm | parser

final_dict = {}
i = 0
while True:
if i == 20:
break
document = random.choice(documents)
context = document.page_content
if len(context) < 30:
continue
questions = chain.invoke({"context" : context})
final_dict[f"batch_{i}"] = questions
i += 1

cleaned_data = {}
i = 1

for batch_key, batch_value in final_dict.items():

for question_key, question_value in batch_value.items():

    cleaned_data[f"question_{i}"] = {
        'question': question_value['question'],
        'answer': question_value['answer']
    }
    i+=1

with open("test_data.json", "w") as file:
json.dump(cleaned_data, file, indent=4)r_store_agentic"
)

vectordb = Chroma(persist_directory="vector_store_agentic", embedding_function=OpenAIEmbeddings(api_key=open_ai_key))

In [26]:
get_llm_response("Is it illegal for a company in India (across india) having less than 1000 employees to not pay my salary after 10th? Also tell me the time limit.", vectordb)[0]

'In India, the Payment of Wages Act, 1936, applies to employees whose wages do not exceed a certain amount (as notified by the government) and ensures timely payment of wages. According to the Act, wages must be paid by the 7th of the following month if the establishment employs less than 1,000 workers. Therefore, it is generally required for companies in India to pay salaries by the 7th of each month for establishments with fewer than 1,000 employees. Not paying salaries by this date would be considered a violation of the Act. It is advisable to check the specific terms of your employment contract and consult with a legal expert for detailed advice.'

In [37]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from config import open_ai_key
import json
import random

llm = ChatOpenAI(model="gpt-4o", api_key=open_ai_key)

QUESTIONS_GENERATOR = """
Given below is a context from one of the HR Policies of India.

context: {context}

Think about 5 questions that a person may ask whose answer can be given from the context

List down 5 questions in json format and json format only. Given below is reference json structure.

{{
    "question_1": {{"question": str, "answer": str}},
    "question_2": {{"question": str, "answer": str}},
    "question_3": {{"question": str, "answer": str}},
    "question_4": {{"question": str, "answer": str}},
    "question_5": {{"question": str, "answer": str}},
}}

The questions you ask, make sure that the answer can be found in the context. Make sure that tha question and answer is relevant. Dont give vague question or answer. Dont give too general questions or answers.
WHile answering the question, name the policy in detail which you are referring to

"""

class Questions(BaseModel):
    question_1: dict = Field(description="First Question")
    question_2: dict = Field(description="Second Question")
    question_3: dict = Field(description="Third Question")
    question_4: dict = Field(description="Fourth Question")
    question_5: dict = Field(description="Fifth Question")

parser = JsonOutputParser(pydantic_object=Questions)

question_prompt = PromptTemplate(
    template=QUESTIONS_GENERATOR,
    input_variables=["context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
    )

chain = question_prompt | llm | parser

import concurrent.futures
import random

final_dict = {}

def process_batch(i):
    while True:
        document = random.choice(documents)
        context = document.page_content
        if len(context) < 30:
            continue
        questions = chain.invoke({"context": context})
        return f"batch_{i}", questions

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(process_batch, i): i for i in range(50)}
    
    for future in concurrent.futures.as_completed(futures):
        batch_key, questions = future.result()
        final_dict[batch_key] = questions
        print(f"complete batch {batch_key}")

complete batch batch_7
complete batch batch_0
complete batch batch_1
complete batch batch_2
complete batch batch_5
complete batch batch_4
complete batch batch_8
complete batch batch_3
complete batch batch_9
complete batch batch_11
complete batch batch_6
complete batch batch_10
complete batch batch_13
complete batch batch_16
complete batch batch_14
complete batch batch_12
complete batch batch_17
complete batch batch_18
complete batch batch_20


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-MleP4eNCIIMjcaTNaMI7A9vQ on tokens per min (TPM): Limit 30000, Used 29559, Requested 785. Please try again in 688ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [39]:

cleaned_data = {}
i = 1

for batch_key, batch_value in final_dict.items():
    for question_key, question_value in batch_value.items():
        cleaned_data[f"question_{i}"] = {
            'question': question_value['question'],
            'answer': question_value['answer']
        }
        i+=1

In [41]:
with open("test_data.json", "w") as file:
    json.dump(cleaned_data, file, indent=4)

In [42]:
import concurrent.futures

llm_response_data = {}

def process_llm_response(k, v):
    """Function to fetch LLM response in parallel."""
    answer, context = get_llm_response(v["question"], vectordb)
    return k, {"question": v["question"], "answer": answer, "context": context}

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(process_llm_response, k, v): k for k, v in cleaned_data.items()}
    
    for future in concurrent.futures.as_completed(futures):
        k, result = future.result()
        llm_response_data[k] = result


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-MleP4eNCIIMjcaTNaMI7A9vQ on tokens per min (TPM): Limit 30000, Used 28497, Requested 2458. Please try again in 1.91s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [44]:
with open("llm_response_data_agentic.json", "w") as file:
    json.dump(llm_response_data, file, indent=4)

In [46]:

import deepeval
import os

os.environ["OPENAI_API_KEY"] = open_ai_key

test_cases = []
for k,v in llm_response_data.items():
    question = v["question"]
    actual_output = v["answer"]
    expected_output = cleaned_data[k]["answer"]
    retrieval_context = v["context"]
    test_case = deepeval.test_case.LLMTestCase(
        input=question,
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context = [retrieval_context]
        )
    test_cases.append(test_case)

deepeval.evaluate(
    test_cases=test_cases,
    metrics=[
    deepeval.metrics.AnswerRelevancyMetric(),
    deepeval.metrics.FaithfulnessMetric(),
    deepeval.metrics.ContextualPrecisionMetric(),
    deepeval.metrics.ContextualRecallMetric(),
    deepeval.metrics.ContextualRelevancyMetric(),
    ],
)

0.7741935483870968 0.6206896551724137 0.5185185185185185 0.7741935483870968
0.72 0.3478260869565218 0.19047619047619047 0.4800000000000001
0.26086956521739124 0.0909090909090909 0.0 0.21739130434782608
0.625 0.3913043478260869 0.27272727272727276 0.5
0.3859649122807018 0.2909090909090909 0.18867924528301888 0.3859649122807018
0.8235294117647058 0.7499999999999999 0.7333333333333334 0.8235294117647058
0.0 0.0 0.0 0.0
0.39999999999999997 0.22222222222222224 0.19672131147540983 0.3384615384615384
0.16666666666666669 0.0 0.0 0.16666666666666669
0.39999999999999997 0.18181818181818182 0.06451612903225806 0.39999999999999997
0.3728813559322034 0.14035087719298248 0.07272727272727274 0.3389830508474576
0.30000000000000004 0.10526315789473684 0.05555555555555555 0.25
0.2978723404255319 0.1333333333333333 0.09302325581395349 0.2978723404255319
0.44 0.16666666666666666 0.08695652173913043 0.4000000000000001
0.37288135593220345 0.10526315789473684 0.03636363636363637 0.2033898305084746
0.33734939

In [64]:
import pandas as pd
from ragas import SingleTurnSample, EvaluationDataset


# Iterate over llm_response_data and populate the DataFrame
data_list = []  # To collect data before creating the DataFrame for efficiency

for k, v in llm_response_data.items():
    row = SingleTurnSample(
        user_input=v["question"],
        retrieved_contexts=[v["context"]],
        response=v["answer"],
        reference=cleaned_data[k]["answer"],
        reference_contexts = [cleaned_data[k]["answer"]]
    )

    data_list.append(row)

In [66]:
dataset = EvaluationDataset(samples=data_list)

In [72]:
from ragas import evaluate
from ragas.metrics import RougeScore, BleuScore, LLMContextPrecisionWithoutReference, LLMContextPrecisionWithReference, LLMContextRecall, ContextEntityRecall, NoiseSensitivity, ResponseRelevancy, NonLLMContextRecall, Faithfulness

result = evaluate(dataset)

# result = evaluate(dataset, metrics=[RougeScore(), 
#                                     BleuScore()
#                                     ])

Evaluating:   4%|▍         | 15/372 [02:48<1:04:21, 10.82s/it]ERROR:ragas.executor:Exception raised in Job[9]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[10]: TimeoutError()
Evaluating:   4%|▍         | 16/372 [03:00<1:06:02, 11.13s/it]ERROR:ragas.executor:Exception raised in Job[12]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[2]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[16]: TimeoutError()
Evaluating:  10%|▉         | 37/372 [05:50<52:49,  9.46s/it]


KeyboardInterrupt: 

ERROR:ragas.executor:Exception raised in Job[34]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[53]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[54]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[55]: AssertionError(set LLM before use)
ERROR:ragas.executor:Exception raised in Job[56]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[57]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[58]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[59]: AssertionError(set LLM before use)
ERROR:ragas.executor:Exception raised in Job[60]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[61]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[62]: AssertionError(LLM is not set)
ERROR:ragas.executor:Exception raised in Job[63]: AssertionError(set LLM before use)
ERROR:ragas.executor:Exception r

In [68]:
result

{'rouge_score': 0.3272, 'bleu_score': 0.1586}